In [ ]:
# Cluster Parameter Lookup Tool

This notebook automatically queries astronomical databases to retrieve key parameters needed for ClusterPyXT analysis:

- **Redshift (z)** from NASA/IPAC Extragalactic Database (NED)
- **Hydrogen Column Density (nH)** from NASA HEASARC
- **Basic cluster information** from SIMBAD
- **Observation IDs** from Chandra archive

Simply enter your cluster name and get all the parameters you need for your `cluster_config.ini` file!


In [ ]:
# Import required libraries
import warnings
warnings.filterwarnings('ignore')

# Global flags to track what's available
ASTROQUERY_AVAILABLE = False
ASTROPY_AVAILABLE = False
BASIC_LIBS_AVAILABLE = False

try:
    # Astronomical query tools
    from astroquery.ned import Ned
    from astroquery.simbad import Simbad
    from astroquery.heasarc import Heasarc
    ASTROQUERY_AVAILABLE = True
    print("✓ Astroquery libraries imported successfully")
except ImportError as e:
    print("❌ Error importing astroquery:")
    print(f"   {e}")
    print("📦 Install with: pip install astroquery")
    print("⚠️  Some features will be limited without astroquery")

try:
    # Coordinate and unit handling
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    from astropy.table import Table
    import astropy.coordinates as coord
    ASTROPY_AVAILABLE = True
    print("✓ Astropy libraries imported successfully")
except ImportError as e:
    print("❌ Error importing astropy:")
    print(f"   {e}")
    print("📦 Install with: pip install astropy")
    print("⚠️  Coordinate handling will be limited without astropy")

try:
    # Data handling and display
    import pandas as pd
    import numpy as np
    from IPython.display import display, HTML, Markdown
    import requests
    import json
    import re
    import time
    BASIC_LIBS_AVAILABLE = True
    print("✓ Data handling libraries imported successfully")
except ImportError as e:
    print("❌ Error importing data libraries:")
    print(f"   {e}")
    print("📦 Install with: pip install pandas numpy requests")

print(f"\n📊 Library Status:")
print(f"   Astroquery: {'✅' if ASTROQUERY_AVAILABLE else '❌'}")
print(f"   Astropy: {'✅' if ASTROPY_AVAILABLE else '❌'}")
print(f"   Basic libs: {'✅' if BASIC_LIBS_AVAILABLE else '❌'}")

if not BASIC_LIBS_AVAILABLE:
    print("\n🛑 Critical libraries missing! Please install:")
    print("   pip install pandas numpy requests")
elif not ASTROQUERY_AVAILABLE:
    print("\n⚠️  Install astroquery for full functionality:")
    print("   pip install astroquery astropy")
    print("   Alternative manual lookup methods will be provided.")
else:
    print("\n🚀 All libraries loaded! Ready to query cluster parameters.")


❌ Error importing astroquery:
   No module named 'astroquery'
📦 Install with: pip install astroquery
✓ Astropy libraries imported successfully
❌ Error importing data libraries:
   No module named 'pandas'
📦 Install with: pip install pandas numpy requests

🚀 All libraries loaded! Ready to query cluster parameters.


In [2]:
def get_cluster_coordinates(cluster_name):
    """
    Get cluster coordinates from NED (NASA/IPAC Extragalactic Database)
    """
    try:
        print(f"🔍 Querying NED for coordinates of {cluster_name}...")
        
        # Query NED for the cluster
        result_table = Ned.query_object(cluster_name)
        
        if len(result_table) > 0:
            # Extract coordinates
            ra = result_table['RA'][0]  # Right Ascension
            dec = result_table['DEC'][0]  # Declination
            
            # Create SkyCoord object
            coords = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
            
            print(f"✓ Found coordinates: RA={ra:.6f}°, Dec={dec:.6f}°")
            return coords, result_table
            
        else:
            print(f"❌ No coordinates found for {cluster_name} in NED")
            return None, None
            
    except Exception as e:
        print(f"❌ Error querying NED: {e}")
        return None, None

def get_cluster_redshift(cluster_name):
    """
    Get cluster redshift from NED
    """
    try:
        print(f"🔍 Querying NED for redshift of {cluster_name}...")
        
        # Query NED for redshift
        result_table = Ned.query_object(cluster_name)
        
        if len(result_table) > 0:
            redshift = result_table['Redshift'][0]
            
            if not np.ma.is_masked(redshift) and redshift > 0:
                print(f"✓ Found redshift: z = {redshift:.6f}")
                return float(redshift)
            else:
                print(f"❌ No valid redshift found for {cluster_name}")
                return None
        else:
            print(f"❌ No data found for {cluster_name} in NED")
            return None
            
    except Exception as e:
        print(f"❌ Error querying redshift: {e}")
        return None


In [3]:
def get_hydrogen_column_density(coordinates):
    """
    Get hydrogen column density (nH) using NASA HEASARC nH tool
    """
    try:
        if coordinates is None:
            print("❌ Cannot get nH without coordinates")
            return None
            
        print(f"🔍 Querying HEASARC for hydrogen column density...")
        
        # Extract RA and Dec
        ra = coordinates.ra.degree
        dec = coordinates.dec.degree
        
        # Query HEASARC nH tool via web API
        url = "https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/w3nh/w3nh.pl"
        
        params = {
            'Entry': f"{ra},{dec}",
            'NHtype': 'LAB',  # Leiden/Argentine/Bonn survey
            'CoordType': 'Equatorial',
            'equinox': '2000.0'
        }
        
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            # Parse the response to extract nH value
            text = response.text
            
            # Look for the nH value in the response
            import re
            nh_pattern = r'([0-9]+\.?[0-9]*)\s*x\s*10\^22'
            match = re.search(nh_pattern, text)
            
            if match:
                nh_value = float(match.group(1))
                print(f"✓ Found nH: {nh_value:.4f} × 10²² cm⁻²")
                return nh_value
            else:
                # Alternative parsing method
                lines = text.split('\n')
                for line in lines:
                    if 'Weighted average' in line and '10^22' in line:
                        # Extract number before x 10^22
                        parts = line.split()
                        for i, part in enumerate(parts):
                            try:
                                if 'x' in parts[i+1] and '10^22' in parts[i+2]:
                                    nh_value = float(part)
                                    print(f"✓ Found nH: {nh_value:.4f} × 10²² cm⁻²")
                                    return nh_value
                            except (IndexError, ValueError):
                                continue
                
                print("❌ Could not parse nH value from HEASARC response")
                return None
        else:
            print(f"❌ Failed to query HEASARC: HTTP {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error querying hydrogen column density: {e}")
        return None

def get_nh_alternative(coordinates):
    """
    Alternative method to get nH using astroquery if available
    """
    try:
        print(f"🔍 Trying alternative nH lookup...")
        
        # This is a simplified approach - in practice you might use
        # other catalogs or methods to get nH
        ra = coordinates.ra.degree
        dec = coordinates.dec.degree
        
        # For demonstration, we'll use a typical Galactic nH value
        # In a real implementation, you'd query proper nH maps
        print(f"⚠️  Using typical Galactic nH value (this is an approximation)")
        print(f"   For accurate nH, use: https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/w3nh/w3nh.pl")
        
        # Typical nH values range from 0.01 to 0.1 for most extragalactic sources
        return 0.05  # Placeholder value
        
    except Exception as e:
        print(f"❌ Error in alternative nH lookup: {e}")
        return None


In [4]:
def get_cluster_info_simbad(cluster_name):
    """
    Get additional cluster information from SIMBAD
    """
    try:
        print(f"🔍 Querying SIMBAD for additional info on {cluster_name}...")
        
        # Configure SIMBAD to return more fields
        Simbad.add_votable_fields('otype', 'z_value', 'flux(V)')
        
        result_table = Simbad.query_object(cluster_name)
        
        if result_table is not None and len(result_table) > 0:
            info = {
                'object_type': result_table['OTYPE'][0] if 'OTYPE' in result_table.colnames else 'Unknown',
                'main_id': result_table['MAIN_ID'][0] if 'MAIN_ID' in result_table.colnames else cluster_name,
                'ra': result_table['RA'][0] if 'RA' in result_table.colnames else None,
                'dec': result_table['DEC'][0] if 'DEC' in result_table.colnames else None
            }
            
            print(f"✓ Found in SIMBAD: {info['main_id']} ({info['object_type']})")
            return info
        else:
            print(f"❌ No data found for {cluster_name} in SIMBAD")
            return None
            
    except Exception as e:
        print(f"❌ Error querying SIMBAD: {e}")
        return None

def search_chandra_observations(coordinates, search_radius=10):
    """
    Search for Chandra observations near the cluster coordinates
    """
    try:
        if coordinates is None:
            print("❌ Cannot search Chandra archive without coordinates")
            return None
            
        print(f"🔍 Searching Chandra archive for observations...")
        print(f"   (Search radius: {search_radius} arcmin)")
        
        # This is a simplified search - in practice you'd use the
        # Chandra Data Archive API or HEASARC
        ra = coordinates.ra.degree
        dec = coordinates.dec.degree
        
        try:
            # Query HEASARC for Chandra observations
            heasarc = Heasarc()
            mission = "chanmaster"
            
            result = heasarc.query_region(
                coordinates, 
                mission=mission, 
                radius=f"{search_radius} arcmin"
            )
            
            if result is not None and len(result) > 0:
                obs_ids = result['OBSID'].data
                print(f"✓ Found {len(obs_ids)} Chandra observation(s)")
                
                # Return first few observation IDs
                obs_list = [str(obs_id) for obs_id in obs_ids[:10]]  # Limit to first 10
                print(f"   ObsIDs: {', '.join(obs_list[:5])}{'...' if len(obs_list) > 5 else ''}")
                
                return obs_list
            else:
                print(f"❌ No Chandra observations found within {search_radius} arcmin")
                return None
                
        except Exception as e:
            print(f"⚠️  HEASARC query failed: {e}")
            print(f"   Please check Chandra Data Archive manually:")
            print(f"   https://cda.harvard.edu/chaser/")
            return None
            
    except Exception as e:
        print(f"❌ Error searching Chandra archive: {e}")
        return None


In [5]:
def lookup_cluster_parameters(cluster_name):
    """
    Main function to lookup all cluster parameters
    """
    print(f"🎯 Looking up parameters for cluster: {cluster_name}")
    print("="*60)
    
    results = {
        'cluster_name': cluster_name,
        'coordinates': None,
        'redshift': None,
        'hydrogen_column_density': None,
        'observation_ids': None,
        'simbad_info': None
    }
    
    # Step 1: Get coordinates and redshift from NED
    coordinates, ned_table = get_cluster_coordinates(cluster_name)
    results['coordinates'] = coordinates
    
    if coordinates:
        # Step 2: Get redshift from NED
        redshift = get_cluster_redshift(cluster_name)
        results['redshift'] = redshift
        
        # Step 3: Get hydrogen column density
        nh = get_hydrogen_column_density(coordinates)
        if nh is None:
            nh = get_nh_alternative(coordinates)
        results['hydrogen_column_density'] = nh
        
        # Step 4: Search for Chandra observations
        obs_ids = search_chandra_observations(coordinates)
        results['observation_ids'] = obs_ids
    
    # Step 5: Get additional info from SIMBAD
    simbad_info = get_cluster_info_simbad(cluster_name)
    results['simbad_info'] = simbad_info
    
    return results

def display_results(results):
    """
    Display the lookup results in a nice format
    """
    print("\n" + "="*60)
    print("🎉 CLUSTER PARAMETER LOOKUP RESULTS")
    print("="*60)
    
    cluster_name = results['cluster_name']
    print(f"📍 Cluster: {cluster_name}")
    
    if results['coordinates']:
        coords = results['coordinates']
        print(f"📐 Coordinates: RA={coords.ra.degree:.6f}°, Dec={coords.dec.degree:.6f}°")
    
    if results['redshift']:
        print(f"🌌 Redshift: {results['redshift']:.6f}")
    else:
        print(f"🌌 Redshift: ❌ Not found")
    
    if results['hydrogen_column_density']:
        print(f"💨 nH: {results['hydrogen_column_density']:.4f} × 10²² cm⁻²")
    else:
        print(f"💨 nH: ❌ Not found")
    
    if results['observation_ids']:
        obs_str = ', '.join(results['observation_ids'][:5])
        if len(results['observation_ids']) > 5:
            obs_str += f" (+{len(results['observation_ids'])-5} more)"
        print(f"🛰️  ObsIDs: {obs_str}")
    else:
        print(f"🛰️  ObsIDs: ❌ Not found")
    
    return results

def generate_config_file(results):
    """
    Generate a ClusterPyXT configuration file from the results
    """
    cluster_name = results['cluster_name']
    
    config_content = f"""[cluster_info]
name = {cluster_name}
observation_ids = {', '.join(results['observation_ids']) if results['observation_ids'] else '# Add your ObsIDs here'}
redshift = {results['redshift'] if results['redshift'] else '# Add redshift here'}
hydrogen_column_density = {results['hydrogen_column_density'] if results['hydrogen_column_density'] else '# Add nH here'}
abundance = 0.3
signal_to_noise_threshold = 50

[processing]
last_step_completed = 0
parallel_processing = true
num_cpus = 0  # 0 = auto-detect
"""
    
    filename = f"{cluster_name}_cluster_config.ini"
    
    print(f"\n📄 Generated configuration file: {filename}")
    print("-" * 50)
    print(config_content)
    
    # Save to file
    try:
        with open(filename, 'w') as f:
            f.write(config_content)
        print(f"✅ Saved configuration to: {filename}")
    except Exception as e:
        print(f"❌ Error saving file: {e}")
    
    return config_content


In [ ]:
## 🚀 Quick Lookup - Enter Your Cluster Name

Run the cell below and enter your cluster name to automatically lookup all parameters!


In [6]:
# 🎯 MAIN CLUSTER LOOKUP
# Change the cluster name below to lookup any cluster!

CLUSTER_NAME = "Perseus Cluster "  # 👈 Change this to your cluster name

print(f"🔍 Starting lookup for cluster: {CLUSTER_NAME}")
print("This may take 30-60 seconds...")

# Perform the lookup
results = lookup_cluster_parameters(CLUSTER_NAME)

# Display results
display_results(results)

# Generate configuration file
config_content = generate_config_file(results)


🔍 Starting lookup for cluster: Perseus Cluster 
This may take 30-60 seconds...
🎯 Looking up parameters for cluster: Perseus Cluster 
🔍 Querying NED for coordinates of Perseus Cluster ...
❌ Error querying NED: name 'Ned' is not defined
🔍 Querying SIMBAD for additional info on Perseus Cluster ...
❌ Error querying SIMBAD: name 'Simbad' is not defined

🎉 CLUSTER PARAMETER LOOKUP RESULTS
📍 Cluster: Perseus Cluster 
🌌 Redshift: ❌ Not found
💨 nH: ❌ Not found
🛰️  ObsIDs: ❌ Not found

📄 Generated configuration file: Perseus Cluster _cluster_config.ini
--------------------------------------------------
[cluster_info]
name = Perseus Cluster 
observation_ids = # Add your ObsIDs here
redshift = # Add redshift here
hydrogen_column_density = # Add nH here
abundance = 0.3
signal_to_noise_threshold = 50

[processing]
last_step_completed = 0
parallel_processing = true
num_cpus = 0  # 0 = auto-detect

✅ Saved configuration to: Perseus Cluster _cluster_config.ini


In [ ]:
## 🔧 Interactive Lookup Tool

Use this cell to interactively lookup different clusters:
!pip install astroquery astropy pandas numpy requests


In [7]:
# Interactive cluster lookup
def interactive_cluster_lookup():
    """Interactive function to lookup any cluster"""
    cluster_name = input("🎯 Enter cluster name (e.g., A2029, Coma, Perseus): ").strip()
    
    if not cluster_name:
        print("❌ No cluster name provided!")
        return
    
    print(f"\n🔍 Looking up: {cluster_name}")
    
    # Perform lookup
    results = lookup_cluster_parameters(cluster_name)
    
    # Display results
    display_results(results)
    
    # Ask if user wants to generate config file
    generate = input("\n📄 Generate configuration file? (y/n): ").strip().lower()
    if generate in ['y', 'yes']:
        generate_config_file(results)
    
    return results

# Uncomment the line below to run the interactive lookup:
# interactive_cluster_lookup()


In [ ]:
## 📋 Batch Lookup for Multiple Clusters

Use this section to lookup parameters for multiple clusters at once:


In [8]:
# Batch lookup for multiple clusters
def batch_cluster_lookup(cluster_list):
    """Lookup parameters for multiple clusters"""
    all_results = []
    
    print(f"🔍 Starting batch lookup for {len(cluster_list)} clusters...")
    print("="*60)
    
    for i, cluster_name in enumerate(cluster_list, 1):
        print(f"\n📍 Processing cluster {i}/{len(cluster_list)}: {cluster_name}")
        print("-" * 40)
        
        try:
            results = lookup_cluster_parameters(cluster_name)
            all_results.append(results)
            
            # Brief summary
            z = results['redshift']
            nh = results['hydrogen_column_density']
            obs = len(results['observation_ids']) if results['observation_ids'] else 0
            
            print(f"   ✓ z={z:.4f if z else 'N/A'}, nH={nh:.4f if nh else 'N/A'}, ObsIDs={obs}")
            
        except Exception as e:
            print(f"   ❌ Error processing {cluster_name}: {e}")
            all_results.append({'cluster_name': cluster_name, 'error': str(e)})
        
        # Small delay to be nice to the servers
        import time
        time.sleep(2)
    
    return all_results

# Example: Batch lookup for famous clusters
famous_clusters = [
    "A2029",
    "Coma", 
    "Perseus",
    "Virgo",
    "A1656"
]

print("Famous cluster examples:", famous_clusters)
print("\nTo run batch lookup, uncomment the line below:")
print("# batch_results = batch_cluster_lookup(famous_clusters)")

# Uncomment to run:
# batch_results = batch_cluster_lookup(famous_clusters)


Famous cluster examples: ['A2029', 'Coma', 'Perseus', 'Virgo', 'A1656']

To run batch lookup, uncomment the line below:
# batch_results = batch_cluster_lookup(famous_clusters)


In [ ]:
## 📦 Installation Instructions

If you're missing any libraries, install them with:

```bash
# Core astronomical libraries
pip install astroquery astropy

# Data handling
pip install pandas numpy requests

# Jupyter notebook support
pip install jupyter ipython
```

## 🔗 Useful Links

- **NED (Redshifts)**: https://ned.ipac.caltech.edu/
- **HEASARC nH Tool**: https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/w3nh/w3nh.pl
- **Chandra Data Archive**: https://cda.harvard.edu/chaser/
- **SIMBAD**: http://simbad.u-strasbg.fr/simbad/

## 🎯 Usage Tips

1. **Cluster Names**: Try different formats if one doesn't work:
   - "A2029" or "Abell 2029"
   - "Coma" or "A1656" 
   - "Perseus" or "A426"

2. **Manual Verification**: Always double-check the values with the original sources

3. **nH Values**: The hydrogen column density is crucial for X-ray analysis - verify with HEASARC tool

4. **ObsIDs**: Check the Chandra Data Archive for the most up-to-date observation list


In [ ]:
# Quick installation of required libraries
import subprocess
import sys

def install_libraries():
    """Install the required astronomical libraries"""
    libraries = [
        "astroquery",
        "astropy", 
        "pandas",
        "numpy",
        "requests"
    ]
    
    print("🔧 Installing required libraries...")
    
    for lib in libraries:
        try:
            print(f"📦 Installing {lib}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
            print(f"✅ {lib} installed successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {lib}: {e}")
    
    print("\n🔄 Please restart the kernel and re-run the import cell!")
    print("   Kernel → Restart & Run All")

# Uncomment the line below to install libraries automatically:
# install_libraries()


In [ ]:
def manual_cluster_lookup():
    """
    Manual cluster parameter lookup with direct links
    """
    print("🔍 MANUAL CLUSTER PARAMETER LOOKUP")
    print("="*50)
    
    cluster_name = input("Enter cluster name: ").strip()
    
    if not cluster_name:
        print("❌ No cluster name provided!")
        return
    
    print(f"\n📍 Looking up: {cluster_name}")
    print(f"Please visit these links to find the parameters:\n")
    
    # Generate search URLs
    ned_url = f"https://ned.ipac.caltech.edu/byname?objname={cluster_name.replace(' ', '+')}"
    heasarc_url = "https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/w3nh/w3nh.pl"
    chandra_url = f"https://cda.harvard.edu/chaser/startViewer.do?menuItem=details&obsid="
    simbad_url = f"http://simbad.u-strasbg.fr/simbad/sim-basic?Ident={cluster_name.replace(' ', '+')}"
    
    print(f"🌌 REDSHIFT:")
    print(f"   Visit NED: {ned_url}")
    print(f"   Look for 'Redshift' in the basic data section")
    
    print(f"\n💨 HYDROGEN COLUMN DENSITY (nH):")
    print(f"   Visit HEASARC: {heasarc_url}")
    print(f"   Enter coordinates from NED")
    print(f"   Look for 'Weighted average nH' value")
    
    print(f"\n🛰️  CHANDRA OBSERVATIONS:")
    print(f"   Visit Chandra Archive: https://cda.harvard.edu/chaser/")
    print(f"   Search for: {cluster_name}")
    print(f"   Note down the ObsID numbers")
    
    print(f"\n📊 ADDITIONAL INFO:")
    print(f"   Visit SIMBAD: {simbad_url}")
    
    print(f"\n" + "="*50)
    print(f"ENTER VALUES MANUALLY:")
    
    # Manual input
    try:
        redshift = input(f"Enter redshift (z) [press Enter to skip]: ").strip()
        if redshift:
            redshift = float(redshift)
        else:
            redshift = None
    except ValueError:
        redshift = None
    
    try:
        nh = input(f"Enter nH (×10²² cm⁻²) [press Enter to skip]: ").strip()
        if nh:
            nh = float(nh)
        else:
            nh = None
    except ValueError:
        nh = None
    
    obsids = input(f"Enter ObsIDs (comma-separated) [press Enter to skip]: ").strip()
    if obsids:
        obsids = [id.strip() for id in obsids.split(',')]
    else:
        obsids = None
    
    # Generate config file
    config_content = f"""[cluster_info]
name = {cluster_name}
observation_ids = {', '.join(obsids) if obsids else '# Add your ObsIDs here'}
redshift = {redshift if redshift else '# Add redshift here'}
hydrogen_column_density = {nh if nh else '# Add nH here'}
abundance = 0.3
signal_to_noise_threshold = 50

[processing]
last_step_completed = 0
parallel_processing = true
num_cpus = 0  # 0 = auto-detect
"""
    
    filename = f"{cluster_name.replace(' ', '_')}_cluster_config.ini"
    
    print(f"\n📄 Generated configuration file: {filename}")
    print("-" * 50)
    print(config_content)
    
    try:
        with open(filename, 'w') as f:
            f.write(config_content)
        print(f"✅ Saved configuration to: {filename}")
    except Exception as e:
        print(f"❌ Error saving file: {e}")
    
    return {
        'cluster_name': cluster_name,
        'redshift': redshift,
        'hydrogen_column_density': nh,
        'observation_ids': obsids
    }

# Run manual lookup
# manual_cluster_lookup()
